In [ ]:
import com.google.gson.GsonBuilder
import hu.akos.hollo.szabo.math.Permutation
import hu.raven.puppet.logic.statistics.edgeHistogramMatrix
import hu.raven.puppet.logic.statistics.mapStates
import hu.raven.puppet.utility.PermutationTypeAdapter
import hu.raven.puppet.model.state.BacteriophageAlgorithmState
import hu.raven.puppet.model.utility.math.GraphEdge
import java.io.File
import java.time.LocalDate
import java.time.LocalDateTime
import hu.raven.puppet.utility.LocalDateTimeTypeAdapter
import hu.raven.puppet.utility.LocalDateTypeAdapter
import com.google.gson.reflect.TypeToken
import hu.raven.puppet.job.experiments.BacteriophageAlgorithmConfiguration

In [ ]:
val gson = GsonBuilder()
    .registerTypeAdapter(Permutation::class.java, PermutationTypeAdapter)
    .registerTypeAdapter(LocalDateTime::class.java, LocalDateTimeTypeAdapter)
    .registerTypeAdapter(LocalDate::class.java, LocalDateTypeAdapter)
    .create()

In [ ]:
val costTimeSeriesFolder = File("D:\\Research\\Results3\\extractions\\costTimeSeries")

In [ ]:
data class TimeSeriesEntry(
    val time: Long,
    val index: Int,
    val value: Int,
)

data class ConfigurationWithTimeSeries(
    val configuration: BacteriophageAlgorithmConfiguration,
    val timeSeries: List<TimeSeriesEntry>
)

val timeSeriesType = object : TypeToken<List<Long>>() {}.type
var records = costTimeSeriesFolder.listFiles()
    .mapNotNull { entryFolder ->
        val configurationFile = File("${entryFolder.path}\\configuration.json")
        val configuration = gson.fromJson<BacteriophageAlgorithmConfiguration>(
            configurationFile.readText(),
            BacteriophageAlgorithmConfiguration::class.java
        )

        if (configuration.fileName != "instance2.json") return@mapNotNull null

        val timeSeriesFile = File("${entryFolder.path}\\timeSeries.json")
        val timeSerie = timeSeriesFile.useLines { lines ->
            lines
                .map {
                    gson.fromJson<List<Long>>(it, timeSeriesType)
                }
                .withIndex()
                .chunked(1000)
                .map { it.last() }
                .toList()
                .let { timeSeries ->
                    timeSeries
                        .asSequence()
                        .withIndex()
                        .filter {
                            it.index == 0 || timeSeries[it.index - 1].value[1] > it.value.value[1]
                        }
                        .map { it.value }
                        .map {
                            TimeSeriesEntry(
                                time = it.value.first(),
                                index = it.index,
                                value = it.value.last().toInt()
                            )
                        }
                        .toList()
                }
        }

        Pair(configuration, timeSerie)
    }
    .toMap()
    .map { ConfigurationWithTimeSeries(it.key, it.value) }



In [ ]:
%use kandy

In [ ]:
val indexLimit = records.minOf {
    it.timeSeries.maxOf { it.index }
}
val timeLimit = records.minOf {
    it.timeSeries.maxOf { it.time }
}

In [ ]:
records.size

In [ ]:
fun <T> valueSequencer(value: T, count: Int) =
    sequence {
        repeat(count) { yield(value) }
    }

plot {
    records
        .asSequence()
        //.filter { it.configuration.mutationPercentage <=0.13 } //pip
        //.filter { it.configuration.cloneCycleCount == 16 }
        //.filter { it.configuration.cloneCount == 16 }
        .forEachIndexed { index, record ->
            line {
                val indexCount = record.timeSeries.count { it.index <= indexLimit }
                color(
                    valueSequencer(
                        record.configuration.mutationPercentage,
                        indexCount
                    ).asIterable()
                ) {
                    scale = continuous(Color.BLUE..Color.RED)
                }
                x(record.timeSeries.map { it.index }.take(indexCount))
                y(record.timeSeries.map { it.value }.take(indexCount))
            }
        }
}